<a target="_blank" href="https://colab.research.google.com/github/HN-Vignolles/Lab/blob/main/notebooks/IDE.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from matplotlib.ticker import EngFormatter

"""Vierquadrantenkennlinienfeld"""

vth = 0.025
Va = 50
n = 1.2
I_S = 5e-12
betaF = 100
betaR = 2
I_S_BE = I_S/betaF
I_S_BC = I_S/betaR
linVbc = np.linspace(-4, 1.2, 100)
linVbe = np.linspace(-1.2, 1.2, 100)
mgVce, mgVbe = np.mgrid[0:4:200j, 0:1:200j]
engI = EngFormatter(places=2, unit='A')
engV = EngFormatter(places=2, unit='V')

def calcIc(Vbe, Vbc):
    return I_S*(np.exp(Vbe/(n*vth))*(1+(Vbe-Vbc)/Va)-np.exp(Vbc/(n*vth))) - I_S_BC*(np.exp(Vbc/(n*vth))-1)

def calcIb(Vbe, Vbc):
    return I_S_BE*(np.exp(Vbe/(n*vth))-1) + I_S_BC*(np.exp(Vbc/(n*vth))-1)

ibLevel = calcIb(mgVbe, -mgVce+mgVbe)

def interactive_plot(Vbe, Vce, Vcc, Rc):
    fig = plt.figure(figsize=(12, 6))
    
    icLim = 0.2
    ibLim = -2e-3
    Vbc = -Vce + Vbe
    Ib = calcIb(Vbe, Vbc)
    Ic = calcIc(Vbe, Vbc)
    ibIcAx = fig.add_subplot(221)
    ibIcAx.plot(-1.0*calcIb(linVbe, Vbc), calcIc(linVbe, Vbc))
    ibIcAx.plot([-1.0*Ib, -1.0*Ib], [0, 3], color="black", linestyle="--", label=f"$I_B = ${engI(Ib)}, $V_{{BE}} = ${engV(Vbe)}")
    ibIcAx.plot([-30e-3, 0], [Ic, Ic], color="black", linestyle="--", label=f"$I_C = ${engI(Ic)}, $V_{{BC}} = ${engV(Vbc)}")
    ibIcAx.set_xlabel(r"$-I_B$")
    ibIcAx.set_ylabel(r"$I_C$")
    ibIcAx.set_xlim(ibLim, 0)
    ibIcAx.set_ylim(0, icLim)
    ibIcAx.xaxis.set_major_formatter(engI)
    ibIcAx.tick_params(axis='x', labelrotation=45)
    ibIcAx.grid(linestyle=":")
    ibIcAx.legend()

    vceIcAx = fig.add_subplot(222)
    vceIcAx.plot(-linVbc + Vbe, calcIc(Vbe, linVbc))
    vceIcAx.plot([Vce, Vce], [0, 3], color="black", linestyle="--")
    vceIcAx.plot([0, 4], [Ic, Ic], color="black", linestyle="--")
    vceIcAx.plot([0, Vcc], [Vcc/(1.0*Rc), 0], color="red", label="Load Line")
    vceIcAx.set_xlabel(r"$V_{CE}$")
    vceIcAx.set_ylabel(r"$I_C$")
    vceIcAx.set_xlim(0, 4)
    vceIcAx.set_ylim(0, icLim)
    vceIcAx.grid(linestyle=":")
    vceIcAx.legend()

    ibVbeAx = fig.add_subplot(223)
    ibVbeAx.plot(-1.0*calcIb(linVbe, Vbc), -linVbe)
    ibVbeAx.plot([-1.0*Ib, -1.0*Ib], [0, -1], color="black", linestyle="--", label=f"$I_B = ${engI(Ib)}")
    ibVbeAx.plot([-30e-3, 0], [-Vbe, -Vbe], color="black", linestyle="--", label=f"$V_{{BE}} = ${engV(Vbe)}")
    ibVbeAx.set_xlabel(r"$-I_B$")
    ibVbeAx.set_ylabel(r"$-V_{BE}$")
    ibVbeAx.set_xlim(ibLim, 0)
    ibVbeAx.set_ylim(-1, 0)
    ibVbeAx.xaxis.set_major_formatter(engI)
    ibVbeAx.tick_params(axis='x', labelrotation=45)
    ibVbeAx.grid(linestyle=":")
    ibVbeAx.legend()

    vceVbeAx = fig.add_subplot(224)
    vceVbeAx.contour(mgVce, -1.0*mgVbe, ibLevel, levels=[Ib])
    vceVbeAx.plot([Vce, Vce], [0, -1], color="black", linestyle="--")
    vceVbeAx.plot([0, 4], [-Vbe, -Vbe], color="black", linestyle="--")
    vceVbeAx.set_xlabel(r"$V_{CE}$")
    vceVbeAx.set_ylabel(r"$-V_{BE}$")
    vceVbeAx.set_xlim(0, 4)
    vceVbeAx.set_ylim(-1, 0)
    vceVbeAx.grid(linestyle=":")

    plt.tight_layout()
    plt.show()

vbe_slider = FloatSlider(min=0, max=0.75, step=0.005, value=0.71, description="Vbe:")
vce_slider = FloatSlider(min=0, max=3, step=0.1, value=0.5, description="Vce:")
vcc_slider = FloatSlider(min=2, max=10, step=0.1, value=10.0, description="Vcc(Load Line):")
rc_slider = IntSlider(min=1, max=1e3, step=10, value=80, description="Rc(Load Line):")
interact(interactive_plot, Vbe=vbe_slider, Vce=vce_slider, Vcc=vcc_slider, Rc=rc_slider)